# Classification

* Roadmap
    * Signal (Put all subjects together)
        * all channel
            * no filter
            * low pass
        * averaged signal
            * no filter
            * low pass
    * Algorithm
        * Linear Model: Logistic Regression, SVM, PA
        * Nonlinear Model: SVM (RBF kernel)
        * Neural Network


* ***WARNING [order of the feature > sample size] ***
    * large epoch
    * down sampliing (FIR filter)
    * averaging 16 channels -> 1 channel


In [24]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
import glob
audio = glob.glob("../data/sample1/audio/*")

In [17]:
import numpy as np
from functions import filtering

def extract_array(path):
    """ Since the experiment data is
    tuple with its string 'tag' such as
        - label: ('label', ndarray)
        - eeg: ('eeg', ndarray)
    extract only ndarray and return them.
    """

    label, eeg = np.load(path).items()
    return label[1], eeg[1]

def combine_data(path):
    # initialize
    data = None
    for p in path:
        label, eeg = extract_array(p)
        if data is None:
            labels, data = label, eeg
        else:
            data = np.concatenate((data, eeg), axis=0)
            labels = np.hstack([labels, label])
    return data, labels


class ParseData(object):
    """Preprocess target data

    - Put all subjects together
    - Split them into 'train' and 'test'

     Parameter
    --------------------------
    data_paths: list of paths of experiment data
    
     Attribute
    --------------------------
    train_data: eeg signal of train data
    train_label: labels of train data
    test_data: eeg signal of test data
    test_label: labels of test data
    """

    def __init__(self, data_paths):
        train_data, train_label, test_data, test_label \
            = self.__parse(data_paths)
        
        self.train_data = train_data
        self.train_label = train_label
        self.test_data = test_data
        self.test_label = test_label
    
    def __parse(self, data_paths):
        # split 'test' and 'train'
        path_test = []
        path_train = []
        for d_ in data_paths:
            if 'test' in d_:
                path_test.append(d_)
            elif 'train' in d_:
                path_train.append(d_)
        # modify
        train_data, train_label = combine_data(path_train)
        test_data, test_label = combine_data(path_test)
        return train_data, train_label, test_data, test_label
    
#     def filtering(self):
        

In [13]:
signals = ParseData(audio)

In [53]:
def data(Data):
    # Frequency
    fs = 512
    # Nyquist frequency
    nyq = fs / 2.0  
    # Cut off frequency
    fil = "l"
    cut_low = 1/10*nyq

    inputs = None
    for data in Data:
        data = data.mean(0)
        data = filtering(fil, data, nyq=nyq, cut_low=cut_low,
                         numtaps=225, sampling_rate=2, ignore=100)
        if inputs is None:
            inputs = data
        else:
            inputs = np.vstack([inputs, data])
    return inputs

X_1 = data(signals.test_data)
X_2 = data(signals.train_data)

In [54]:
y_1 = signals.test_label
y_2 = signals.train_label

In [56]:
print("Data:", X_1.shape, X_2.shape)
print("Label:", y_1.shape, y_2.shape)

Data: (8250, 257) (4000, 257)
Label: (8250,) (4000,)


# Simple Classifier

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.utils import shuffle

In [58]:
X_1, y_1 = shuffle(X_1, y_1, random_state=0)
X_2, y_2 = shuffle(X_2, y_2, random_state=0)

In [62]:
clf = {
    "log": LogisticRegression(),
    "svm": SVC(),
}
for clf_ in clf:
    print(clf_)
    
    clf_.fit(X_1, y_1)
    print(clf.score(X_2, y_2))

ValueError: too many values to unpack (expected 2)

In [64]:
clf.items()

dict_items([('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)), ('log', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])